# Modeling Pipeline
Split, train models, and evaluate decisions.


In [ ]:
def split_train_test_by_time(modeling_panel, feature_columns, train_fraction=0.80):
    available_months = np.array(sorted(modeling_panel["month_end"].unique()))
    if len(available_months) < 5:
        raise ValueError("Not enough monthly observations for a time-based split.")

    split_index = int(len(available_months) * train_fraction) - 1
    split_month = available_months[split_index]

    train_mask = modeling_panel["month_end"] <= split_month
    test_mask = modeling_panel["month_end"] > split_month

    train_features = modeling_panel.loc[train_mask, feature_columns]
    train_target = modeling_panel.loc[train_mask, "target_return"]
    test_features = modeling_panel.loc[test_mask, feature_columns]
    test_target = modeling_panel.loc[test_mask, "target_return"]

    if train_features.empty or test_features.empty:
        raise ValueError("Train/test split produced an empty set.")

    return split_month, train_features, train_target, test_features, test_target


In [ ]:
def train_models(train_features, train_target):
    linear_model = Pipeline([
        ("scaler", StandardScaler()),
        ("model", LinearRegression()),
    ])
    random_forest_model = RandomForestRegressor(random_state=42, n_jobs=1)

    linear_model.fit(train_features, train_target)
    random_forest_model.fit(train_features, train_target)

    return linear_model, random_forest_model


In [ ]:
def predict_returns(linear_model, random_forest_model, test_features):
    linear_predictions = pd.Series(
        linear_model.predict(test_features),
        index=test_features.index,
        name="linear_prediction",
    )

    random_forest_predictions = pd.Series(
        random_forest_model.predict(test_features),
        index=test_features.index,
        name="random_forest_prediction",
    )

    return linear_predictions, random_forest_predictions


In [ ]:
def evaluate_threshold(actual_returns, approve_mask):
    return {
        "approved_count": int(approve_mask.sum()),
        "rejected_count": int((~approve_mask).sum()),
        "type_I_errors": int((approve_mask & (actual_returns < 0)).sum()),
        "type_II_errors": int((~approve_mask & (actual_returns >= 0)).sum()),
        "approved_return_sum": float(actual_returns[approve_mask].sum()),
        "rejected_return_sum": float(actual_returns[~approve_mask].sum()),
        "approved_return_avg": float(actual_returns[approve_mask].mean()) if approve_mask.any() else np.nan,
        "rejected_return_avg": float(actual_returns[~approve_mask].mean()) if (~approve_mask).any() else np.nan,
    }


In [ ]:
def evaluate_model_decisions(actual_returns, linear_predictions, random_forest_predictions, thresholds):
    decision_records = []
    flip_records = []

    observation_count = len(actual_returns)
    for threshold in thresholds:
        linear_approve = linear_predictions >= threshold
        random_forest_approve = random_forest_predictions >= threshold

        disagreement_mask = linear_approve != random_forest_approve
        flip_records.append({
            "threshold": threshold,
            "flip_rate": disagreement_mask.mean() if observation_count else np.nan,
            "flips_total": int(disagreement_mask.sum()),
            "linear_approve_rf_reject": int((linear_approve & ~random_forest_approve).sum()),
            "rf_approve_linear_reject": int((~linear_approve & random_forest_approve).sum()),
        })

        for model_name, approve_mask in [
            ("LinearRegression", linear_approve),
            ("RandomForest", random_forest_approve),
        ]:
            metrics = evaluate_threshold(actual_returns, approve_mask)
            metrics.update({"model": model_name, "threshold": threshold})
            decision_records.append(metrics)

    decision_results = pd.DataFrame(decision_records)
    flip_results = pd.DataFrame(flip_records)
    return decision_results, flip_results


In [ ]:
def build_summary_tables(decision_results, flip_results):
    ordered_results = decision_results.sort_values(["threshold", "model"]).reset_index(drop=True)

    detailed_column_order = [
        "model",
        "threshold",
        "approved_count",
        "rejected_count",
        "type_I_errors",
        "type_II_errors",
        "approved_return_sum",
        "rejected_return_sum",
        "approved_return_avg",
        "rejected_return_avg",
    ]
    ordered_results = ordered_results[detailed_column_order]

    flip_summary_table = flip_results.copy()
    flip_summary_table["flip_rate"] = flip_summary_table["flip_rate"].round(4)

    decision_error_table = (
        ordered_results
        .pivot(
            index="threshold",
            columns="model",
            values=["approved_count", "rejected_count", "type_I_errors", "type_II_errors"],
        )
        .sort_index(axis=1, level=[0, 1])
    )

    impact_table = (
        ordered_results
        .pivot(
            index="threshold",
            columns="model",
            values=["approved_return_sum", "rejected_return_sum"],
        )
        .sort_index(axis=1, level=[0, 1])
        .round(6)
    )

    average_impact_table = (
        ordered_results
        .pivot(
            index="threshold",
            columns="model",
            values=["approved_return_avg", "rejected_return_avg"],
        )
        .sort_index(axis=1, level=[0, 1])
        .round(6)
    )

    return ordered_results, flip_summary_table, decision_error_table, impact_table, average_impact_table


In [ ]:
def run_modeling_pipeline(modeling_panel):
    split_month, train_features, train_target, test_features, test_target = split_train_test_by_time(
        modeling_panel,
        MODEL_FEATURE_COLUMNS,
    )

    linear_model, random_forest_model = train_models(train_features, train_target)
    linear_predictions, random_forest_predictions = predict_returns(
        linear_model,
        random_forest_model,
        test_features,
    )

    decision_results, flip_results = evaluate_model_decisions(
        test_target,
        linear_predictions,
        random_forest_predictions,
        DECISION_THRESHOLDS,
    )

    (
        detailed_results,
        flip_summary_table,
        decision_error_table,
        impact_table,
        average_impact_table,
    ) = build_summary_tables(decision_results, flip_results)

    print(f"Train rows: {len(train_features)} | Test rows: {len(test_features)}")
    print(f"Split month: {pd.to_datetime(split_month).date()}")

    print("Table 1. Flip Summary by Threshold")
    display(flip_summary_table)

    print("Table 2. Decision Counts and Error Counts (Side-by-Side)")
    display(decision_error_table)

    print("Table 3. Economic Impact (Realized Return Sums, Side-by-Side)")
    display(impact_table)

    print("Table 4. Average Realized Returns (Side-by-Side)")
    display(average_impact_table)

    print("Detailed Long-Format Results")
    display(detailed_results)

    return {
        "split_month": split_month,
        "train_features": train_features,
        "train_target": train_target,
        "test_features": test_features,
        "test_target": test_target,
        "linear_model": linear_model,
        "random_forest_model": random_forest_model,
        "linear_predictions": linear_predictions,
        "random_forest_predictions": random_forest_predictions,
        "detailed_results": detailed_results,
        "flip_summary_table": flip_summary_table,
        "decision_error_table": decision_error_table,
        "impact_table": impact_table,
        "average_impact_table": average_impact_table,
    }
